<a href="https://colab.research.google.com/github/Ezra-11/knowledge-level-of-wikipedia/blob/main/DataMining_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Business Understanding

## 1.1 Problem Statement
Wikipedia content about *Zambia* remains significantly underrepresented, with very few comprehensive pages dedicated to Zambian topics, history, culture, and current affairs. The *DataLab Research group* at The University of Zambia has identified that understanding the knowledge levels and expertise of current Wikipedia contributors working on Zambian content is crucial for developing targeted strategies to improve content quality and quantity.

Currently, there is no systematic way to assess the expertise and knowledge depth of contributors to Zambian Wikipedia pages, making it difficult to:

- Identify subject matter experts who could mentor new contributors  
- Match contributors with appropriate content areas based on their expertise  
- Develop targeted training programs for contributors with different knowledge levels  
- Optimize content review and quality assurance processes  

*Core Problem:* How can we systematically classify and understand the knowledge levels of Wikipedia contributors working on Zambian content to improve the overall quality and quantity of Wikipedia pages about Zambia?

# New Section

## 1.2 Business Objectives

### Primary Objectives:
- *Enhance Content Quality:* Improve the accuracy, depth, and comprehensiveness of Wikipedia articles about Zambia by better understanding contributor expertise levels  
- *Optimize Contributor Engagement:* Develop targeted strategies to engage contributors based on their knowledge levels and areas of expertise  
- *Facilitate Knowledge Transfer:* Identify expert contributors who can mentor newcomers and provide guidance on complex topics  
- *Improve Content Coverage:* Strategically assign content creation and improvement tasks based on contributor knowledge levels  



 Secondary Objectives:
- **Build Community Capacity:** Create a framework for assessing and developing Wikipedia contributor skills within the Zambian context  
- ***Inform Training Programs:*** Develop evidence-based training curricula tailored to different contributor knowledge levels  
- ***Support Academic Research:*** Provide insights into digital knowledge creation patterns in developing countries  

### Success Metrics:
- *Short-term:* Accurate classification of contributor knowledge levels with measurable confidence scores  
- *Medium-term:* Increased quality scores of Zambian Wikipedia articles (measured by completeness, references, and peer ratings)  
- ***Long-term:*** Growth in the number of active Zambian Wikipedia contributors and articles about Zambia  





##**1.4 Project Success Criteria**

###Model Performance Criteria
- *Primary Metric:* Classification accuracy of at least *80%* on a held-out test set  
- *Precision and Recall:* Minimum *75%* precision and recall for each knowledge level category  
- *F1-Score:* Overall weighted F1-score of at least *0.78*  
- *Cross-validation Stability:* Model performance should not vary by more than *5%* across different data splits  

###Business Impact Criteria
- *Expert Validation:* At least *85%* agreement between model classifications and manual assessments by domain experts  
- *Actionability:* Classification results must provide clear, actionable insights for contributor development strategies  
- *Scalability:* Model should be able to process new contributors and updates efficiently (*< 1 minute* per contributor)  

###Technical Criteria
- *Interpretability:* Model decisions must be explainable with feature importance scores and decision reasoning  
- *Robustness:* Model should maintain performance when applied to contributors from different time periods  
- *Generalizability:* Framework should be adaptable to other developing country contexts or subject areas  

###Data Quality Criteria
- *Coverage:* Successfully classify at least *90%* of active contributors to Zambian Wikipedia pages  
- *Consistency:* Inter-rater reliability (Kappa score) *> 0.7* for manual validation labels  
- *Completeness:* Less than *10%* missing data across critical features  

###Deployment Criteria
- *Integration:* Model should be implementable within existing Wikipedia contributor management workflows  
- *User Acceptance:* Positive feedback from at least *80%* of Wikipedia administrators and experienced contributors who test the system  
- *Maintenance:* Established process for model updates and performance monitoring over time

##**1.3 Data Mining Goals**

Primary Data Mining Goal:
Build a *multi-class classification model* that can automatically categorize Wikipedia contributors working on Zambian content into distinct knowledge level categories (e.g., Novice, Intermediate, Advanced, Expert) based on their contribution patterns, edit quality, and content expertise.

###Specific Technical Objectives

Feature Engineering
Extract meaningful features from contributor data including:
- Edit frequency and consistency patterns  
- Quality indicators (citations added, grammar improvements, factual accuracy)  
- Content complexity (technical terms, specialized knowledge areas)  
- Collaboration patterns (talk page participation, peer interactions)  
- Longevity and commitment indicators  

###Classification Model Development
- Implement and compare multiple classification algorithms (Random Forest, SVM, Neural Networks, etc.)  
- Develop ensemble methods to improve classification accuracy  
- Create interpretable models that explain why a contributor is classified at a particular level  

###Clustering Analysis (Secondary)
- Identify natural groupings of contributors based on their expertise areas and contribution styles  
- Discover contributor archetypes that may not align with traditional knowledge level categories  

###Predictive Modeling
- Predict potential contributor knowledge growth trajectories  
- Identify contributors likely to become long-term, high-quality contributors

# Data Understanding


In [1]:
!pip install requests pandas


In [11]:
# --- Imports ---
import requests
import pandas as pd
from collections import defaultdict
from datetime import datetime
import time

# --- Wikipedia API endpoint ---
API_URL = "https://en.wikipedia.org/w/api.php"

# --- Fetch pages under a category ---
def get_category_members(category, limit=50):
    params = {
        "action": "query",
        "list": "categorymembers",
        "cmtitle": f"Category:{category}",
        "cmlimit": limit,
        "format": "json"
    }
    response = requests.get(API_URL, params=params)
    data = response.json()
    # Only include main articles (ns=0)
    return [page['title'] for page in data['query']['categorymembers'] if page['ns'] == 0]

# --- Fetch revisions (contributors) for a page ---
def get_page_contributors(page_title, limit=100):
    params = {
        "action": "query",
        "prop": "revisions",
        "titles": page_title,
        "rvprop": "user|timestamp|size|comment",
        "rvlimit": limit,
        "format": "json"
    }
    response = requests.get(API_URL, params=params)
    data = response.json()

    contributors = []
    for page_id in data['query']['pages']:
        page = data['query']['pages'][page_id]
        if "revisions" in page:
            for rev in page["revisions"]:
                contributors.append({
                    "user": rev["user"],
                    "page": page["title"],
                    "timestamp": rev["timestamp"],
                    "size": rev.get("size", 0),
                })
    return contributors

# --- Aggregate contributor-level features ---
def aggregate_contributors(pages):
    contributor_stats = defaultdict(lambda: {
        "Total_Edits": 0,
        "Pages_Edited": set(),
        "Total_Size": 0,
        "First_Edit": None,
        "Last_Edit": None
    })

    for page in pages:
        revisions = get_page_contributors(page)
        for rev in revisions:
            user = rev["user"]
            contributor_stats[user]["Total_Edits"] += 1
            contributor_stats[user]["Pages_Edited"].add(rev["page"])
            contributor_stats[user]["Total_Size"] += rev["size"]

            ts = datetime.fromisoformat(rev["timestamp"].replace("Z", "+00:00"))
            if contributor_stats[user]["First_Edit"] is None or ts < contributor_stats[user]["First_Edit"]:
                contributor_stats[user]["First_Edit"] = ts
            if contributor_stats[user]["Last_Edit"] is None or ts > contributor_stats[user]["Last_Edit"]:
                contributor_stats[user]["Last_Edit"] = ts

        # Polite delay to avoid hitting API limits
        time.sleep(0.5)

    # Convert to DataFrame with activity metrics
    rows = []
    for user, stats in contributor_stats.items():
        first = stats["First_Edit"]
        last = stats["Last_Edit"]
        active_days = (last - first).days if first and last else 0
        active_years = active_days / 365 if active_days > 0 else 0
        edits_per_month = stats["Total_Edits"] / (active_days/30) if active_days > 30 else stats["Total_Edits"]

        rows.append({
            "Contributor": user,
            "Total_Edits": stats["Total_Edits"],
            "Pages_Edited": len(stats["Pages_Edited"]),
            "Total_Size": stats["Total_Size"],
            "First_Edit": first,
            "Last_Edit": last,
            "Active_Days": active_days,
            "Activity_Span_Years": round(active_years, 2),
            "Edits_per_Month": round(edits_per_month, 2)
        })

    return pd.DataFrame(rows)

# --- Run pipeline in Colab ---
zambia_pages = get_category_members("Zambia", limit=20)  # fetch first 20 pages for testing
df = aggregate_contributors(zambia_pages)

# Display neatly in Colab
from IPython.display import display
display(df.head(10))

# Save dataset
df.to_csv("zambia_contributors_summary.csv", index=False)
print(f"Dataset saved with {len(df)} rows")


,Contributor,Total_Edits,Pages_Edited,Total_Size,First_Edit,Last_Edit,Active_Days,Activity_Span_Years,Edits_per_Month
0,Kosmopolitní,3,1,455820,2025-03-21 08:05:23+00:00,2025-08-13 06:32:53+00:00,144,0.39,0.62
1,102.208.220.201,1,1,151954,2025-08-11 15:35:23+00:00,2025-08-11 15:35:23+00:00,0,0.00,1.00
2,Guest88,1,1,151954,2025-08-11 13:18:40+00:00,2025-08-11 13:18:40+00:00,0,0.00,1.00
3,Remsense,5,1,755201,2025-01-23 16:22:18+00:00,2025-08-10 00:53:36+00:00,198,0.54,0.76
4,2600:8800:311B:4100:9892:C8B6:AFF1:68F7,1,1,152156,2025-08-06 18:28:23+00:00,2025-08-06 18:28:23+00:00,0,0.00,1.00
5,2001:4479:900A:2F00:78FD:7EE:28B7:95FA,1,1,152155,2025-08-04 23:49:33+00:00,2025-08-04 23:49:33+00:00,0,0.00,1.00
6,Tumbuka Arch,2,1,304021,2025-05-21 15:53:08+00:00,2025-07-15 20:12:05+00:00,55,0.15,1.09
7,Lightoil,1,1,151978,2025-07-14 23:43:14+00:00,2025-07-14 23:43:14+00:00,0,0.00,1.00
8,GeographicAccountant,1,1,151990,2025-07-14 20:10:30+00:00,2025-07-14 20:10:30+00:00,0,0.00,1.00
9,102.149.117.245,1,1,151996,2025-07-14 18:20:23+00:00,2025-07-14 18:20:23+00:00,0,0.00,1.00


Dataset saved with 128 rows
